Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

# データの準備
---

このリポジトリでは、[Azure Open Datasets](https://azure.microsoft.com/services/open-datasets/) のシミュレートされたオレンジ ジュースの売上データを使用して、Azure Machine Learning で Many models をトレーニングし、予測するプロセスを説明します。

このノートブックでは、このソリューション アクセラレータのデータを構成するために必要なすべての手順について説明します:

1. サンプル データのダウンロード
2. トレーニング/予測セットの分割
3. ワークスペースに接続し、データをデータストアにアップロード

### 前提条件
[00_Setup_AML_Workspace](00_Setup_AML_Workspace.ipynb) ノートブックを既に実行している場合は、すべて設定済みです。

## 1.0 サンプル データのダウンロード

この例で使用される時系列データは、シカゴ大学によるドミニク・ファイン・フーズ・データセットに基づいてシミュレートされ、個々の店舗で3つの異なるオレンジジュースブランドの2年間の売上に着目しています。データセットの詳細については、[こちら](https://azure.microsoft.com/services/open-datasets/catalog/sample-oj-sales-simulated/)をご覧ください。

完全なデータセットには、3つのオレンジジュースブランドを持つ3,991店舗のシミュレートされた売上が含まれており、11,973のモデルが多くのモデルパターンの力を発揮できるようにトレーニングすることができます。各時系列データには、'1990-06-14'から'1992-10-01'までのデータが含まれています。

データをダウンロードするには、`azureml-opendatasets` パッケージが必要です。次の方法でインストールできます:

In [ ]:
%pip install azureml-opendatasets

In [ ]:
%pip install update azureml-opendatasets==1.10.0

最初の 10 個のファイルをダウンロードするところから始めますが、以下のコードを編集して、簡単に11,973 モデルすべてをトレーニングできます。

In [ ]:
dataset_maxfiles = 10 # 11973 または 0 に設定すると、すべてのファイルを取得

In [ ]:
import os
from azureml.opendatasets import OjSalesSimulated

# 全てのデータをプル
oj_sales_files = OjSalesSimulated.get_file_dataset()

# `dataset_maxfiles` で指定した最初のファイル数のみプル
if dataset_maxfiles:
    oj_sales_files = oj_sales_files.take(dataset_maxfiles)

# ダウンロード先のフォルダを作成
target_path = 'oj_sales_data' 
os.makedirs(target_path, exist_ok=True)

# データのダウンロード
oj_sales_files.download(target_path, overwrite=True)

## 2.0 データを2 つのセットに分割

次に、各データセットを 2 つに分割します。トレーニングファイルには'1992-5-28'以前のデータレコードが含まれ、各時系列データの残りの部分は推論ファイルに保存されます。

最後に、両方のデータ ファイルのセットをワークスペースの既定の[データストア](https://docs.microsoft.compython/api/azureml-core/azureml.core.datastore(class))にアップロードします。

In [ ]:
from scripts.helper import split_data

# 推論データセットに分割する日付と、データ内のタイムスタンプ列の名前を指定
timestamp_column = 'WeekStarting'
split_date = '1992-05-28'

# 各ファイルを分割し、対応するディレクトリに格納
train_path, inference_path = split_data(target_path, timestamp_column, split_date)

## 3.0 AML ワークスペースのデータストアへのデータのアップロード

[ノートブックのセットアップ](00_Setup_AML_Workspace.ipynb)で[ワークスペース](https://docs.microsoft.com/ja-jp/python/api/azureml-core/azureml.core.workspace.workspace?view=azure-ml-py)を作成しました。ここでは、その環境にデータを登録します。

In [ ]:
from azureml.core.workspace import Workspace

ws = Workspace.from_config()

# ワークスペースを確認
ws.get_details()

両方のデータ ファイルのセットがワークスペースのデフォルトの[データストア](https://docs.microsoft.com/ja-jp/azure/machine-learning/how-to-access-data)にアップロードされます。データストアは、トレーニングや予測のためにデータを保存、アクセスできる場所です。データストアからデータにアクセスする方法については、[データストアのドキュメント](https://docs.microsoft.com/ja-jp/python/api/azureml-core/azureml.core.datastore(class)?view=azure-ml-py)を参照してください。

In [ ]:
# デフォルトのデータストアに接続
datastore = ws.get_default_datastore()

# トレーニングデータのアップロード
ds_train_path = target_path + '_train'
datastore.upload(src_dir=train_path, target_path=ds_train_path, overwrite=True)

# 推論データのアップロード
ds_inference_path = target_path + '_inference'
datastore.upload(src_dir=inference_path, target_path=ds_inference_path, overwrite=True)

### *[オプション]* データが既に Azure にある場合: そこからデータストアを作成する

データが既に Azure に存在する場合は、ローカル コンピューターから既定のデータストアにアップロードする必要はありません。代わりに、そのデータのセットを参照する新しいデータストアを作成できます。次に、サンプル データが格納されている BLOB ストレージ内のコンテナーからデータストアを設定する方法の例を示します。

この場合、オレンジジュースデータは、以下の情報で定義されたパブリック BLOB コンテナーで使用できます。この場合は、アカウントの資格情報も指定する必要があります。詳細については、[このドキュメント](https://docs.microsoft.com/ja-jp/python/api/azureml-core/azureml.core.datastore.datastore?view=azure-ml-py#register-azure-blob-container-workspace--datastore-name--container-name--account-name--sas-token-none--account-key-none--protocol-none--endpoint-none--overwrite-false--create-if-not-exists-false--skip-validation-false--blob-cache-timeout-none--grant-workspace-access-false--subscription-id-none--resource-group-none-)を参照してください。

In [ ]:
blob_datastore_name = "automl_many_models"
container_name = "automl-sample-notebook-data"
account_name = "automlsamplenotebookdata"

In [ ]:
from azureml.core import Datastore

datastore = Datastore.register_azure_blob_container(
    workspace=ws, 
    datastore_name=blob_datastore_name, 
    container_name=container_name,
    account_name=account_name,
    create_if_not_exists=True
)

if 0 < dataset_maxfiles < 11973:
    ds_train_path = 'oj_data_small/'
    ds_inference_path = 'oj_inference_small/'
else:
    ds_train_path = 'oj_data/'
    ds_inference_path = 'oj_inference/'

## 4.0 AML ワークスペースにデータセットを登録する

最後の手順は、トレーニング セットと推論セット用の[データセット](https://docs.microsoft.com/ja-jp/azure/machine-learning/concept-data#datasets)を Azure Machine Learning で作成および登録することです。

現在、[FileDataset](https://docs.microsoft.com/ja-jp/python/api/azureml-core/azureml.data.file_dataset.filedataset?view=azure-ml-py) を使用することが多くのモデル パターンを利用する最良の方法であるため、次のセルで FileDataset を作成します。その後、ワークスペースに FileDataset を[登録](https://docs.microsoft.com/ja-jp/azure/machine-learning/how-to-create-register-datasets#register-datasets)します。これにより、トレーニング/推論セットは、後でモデルをトレーニングして予測を生成するときに簡単に参照できる単純な名前に関連付けられます。

In [ ]:
from azureml.core.dataset import Dataset

# FileDatasetsの作成
ds_train = Dataset.File.from_files(path=datastore.path(ds_train_path), validate=False)
ds_inference = Dataset.File.from_files(path=datastore.path(ds_inference_path), validate=False)

# FileDatasetsの登録
dataset_name = 'oj_data_small' if 0 < dataset_maxfiles < 11973 else 'oj_data'
train_dataset_name = dataset_name + '_train'
inference_dataset_name = dataset_name + '_inference'
ds_train.register(ws, train_dataset_name, create_new_version=True)
ds_inference.register(ws, inference_dataset_name, create_new_version=True)

## 5.0 *[オプション]* 登録されたデータセットの操作

データを登録した後、下記のコマンドを使って簡単に呼び出すことができます。これは、データセットが今後のノートブックでアクセスされる方法です。

In [ ]:
oj_ds = Dataset.get_by_name(ws, name=train_dataset_name)
oj_ds

登録されたデータセットからデータをダウンロードすることもできます:

In [ ]:
download_paths = oj_ds.download()
download_paths

データ ファイルの 1 つを読み込んで、フォーマットを確認してみましょう:

In [ ]:
import pandas as pd

sample_data = pd.read_csv(download_paths[0])
sample_data.head(10)

## 次のステップ

データセットを作成したので、トレーニング ノートブックの 1 つに移動してモデルをトレーニングし、スコアを付ける準備が整いました:

- 自動化ML: [02_AutoML_Training_Pipeline.ipynb](Automated_ML/02_AutoML_Training_Pipeline/02_AutoML_Training_Pipeline.ipynb) を開いてください。
- カスタム スクリプト: [02_CustomScript_Training_Pipeline.ipynb](Custom_Script/02_CustomScript_Training_Pipeline.ipynb) を開いてください。